In [156]:
import json
import gensim.models as g
import jieba
import re
jieba.add_word("马嘉祺公益祺迹",True)
jieba.add_word("新型冠状病毒",True)
stop_word_list = ["是", "的" ,"因为","了","也"]
    
def read_data():
    x = []
    y = []
    y_map = {"happy":1 , "sad" :2 , "angry" : 3 , "neural" : 4, "fear" : 5 , "surprise" : 6}
    with open("virus_train.txt","r",encoding="utf-8") as f: 
        data = f.read()
    data = json.loads(data)
    for i in data:
        temp_x = []
        s = re.sub("//@.*?:","", i["content"])
        s = re.sub("(ht|f)tp(s?)\:\/\/[0-9a-zA-Z]([-.\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%$#_]*)?","",s)
        s = re.sub("[\[\]_.!！+-=——,$%·^『』～；\"，“”。?？＋、： ％~@#￥%……&*《》<>「」{}【】()/]","",s)
        segment_list = jieba.cut(s,cut_all=False)
        
        segment = ",".join(segment_list)
        if s is None:
            continue
        x.append(segment.split(","))
        y.append(y_map[i["label"]])
    return  x , y
x , y = read_data()
# print(x,y)
# print(len(x),len(y))

In [70]:
def gensim_test():
    model = g.Word2Vec(x,min_count=3,vector_size=100)
    print(model)
    
    


In [161]:
word2vec = g.Word2Vec(x, 
                       vector_size=100,   # 词向量维度
                       min_count=1,      # 最小词频, 因为数据量较小, 这里卡1
                       epochs=100) 

# print(word2vec.wv.most_similar("武汉"))

In [162]:
word2vec.wv.most_similar("武汉")
# word2vec.wv.most_similar("天使")
# for i ,v in word2vec.wv.key_to_index.items():
#     print(i,v  )
# word2vec.wv["的"]

[('湖北', 0.4733487367630005),
 ('在', 0.4005740284919739),
 ('的', 0.3984314799308777),
 ('我们', 0.39034581184387207),
 ('了', 0.35161668062210083),
 ('都', 0.34879183769226074),
 ('病例', 0.34381434321403503),
 ('海南', 0.3384853005409241),
 ('你', 0.3307299315929413),
 ('也', 0.32444286346435547)]

In [68]:
a = "//@这瓜沙沙雕雕的保甜://@菜刀曦曦:武汉正在玩儿命求援，请求民间援助。新闻联播在播这个。我不是武汉人看了都有点生气。武汉人应该在家骂大街吧。 http://t.cn/A6PZCMJS"
s = re.sub("//@.*?:",""  , a)
s = re.sub("(ht|f)tp(s?)\:\/\/[0-9a-zA-Z]([-\".\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%$#_]*)?","",s)
s = re.sub("[\[\]\|_.→!+-=——,$%^，（～；。）？、~@#￥%……&*《》<>「」{}【】()/]","",s)
print(s)

武汉正在玩儿命求援请求民间援助新闻联播在播这个我不是武汉人看了都有点生气武汉人应该在家骂大街吧 


In [127]:
a = [["哈哈"],["你是谁啊","玩儿命求援","猫", "狗"],["我是中国人"],["你是什么人","呢"]]
word2vec = g.Word2Vec(a, 
                   vector_size=64,   # 词向量维度
                   min_count=1,      # 最小词频, 因为数据量较小, 这里卡1
                   epochs=1) 
word2vec.wv.most_similar("猫")
word2vec.wv.key_to_index


{'呢': 0,
 '你是什么人': 1,
 '我是中国人': 2,
 '狗': 3,
 '猫': 4,
 '玩儿命求援': 5,
 '你是谁啊': 6,
 '哈哈': 7}